# SQL 1

This notebook provides commands from the SQL1 lecture

## Initialization

Run the next cell to set up PostgreSQL

In [ ]:
# install
!pip install psycopg2-binary
!apt install postgresql postgresql-contrib &>log

## Connect to a database 


In [ ]:
# Set your database configuation information
# set this value to your database's username
dbuser = "netId"
# set this value to your database's name
dbName = "netIddb"
# set this value to your database's port
port = 5432
# set this value to your database's endpoint
endpoint = "postgres.clear.rice.edu"


In [ ]:
# build the connection string
def make_conn_str(dbuser, password, endpoint, port, dbName):

    return f"postgresql+psycopg2://{dbuser}:{password}@{endpoint}:{port}/{dbName}"

        
import getpass
password = getpass.getpass()

In [ ]:
# set connection
%load_ext sql
conn_str = make_conn_str(dbuser, password,endpoint, port, dbName)
# Limit queries to 100 results. Increase this value if needed, but recognize that your JN will increase in size as well. 
%config SqlMagic.displaylimit=100
%sql $conn_str

### Create the tables


In [ ]:
%%sql
DROP TABLE IF EXISTS Frequents;
CREATE TABLE Frequents
(
    drinker VARCHAR(50) NOT NULL,
    cafe VARCHAR(50) NOT NULL,
    PRIMARY KEY (drinker, cafe)
);

DROP TABLE IF EXISTS Likes;
CREATE TABLE Likes
(
    drinker VARCHAR(50) NOT NULL,
    coffee VARCHAR(50) NOT NULL,
    PRIMARY KEY (drinker, coffee)
);


DROP TABLE IF EXISTS Serves;
CREATE TABLE Serves
(
    cafe VARCHAR(50) NOT NULL,
    coffee VARCHAR(50)  NOT NULL,
    PRIMARY KEY (cafe, coffee)
);

DROP TABLE IF EXISTS Rates;
CREATE TABLE Rates
(
    drinker VARCHAR(50) NOT NULL,
    coffee VARCHAR(50) NOT NULL,
    score INTEGER NOT NULL
);


DROP TABLE IF EXISTS Enroll;
DROP TABLE IF EXISTS Student;
DROP TABLE IF EXISTS Course;
DROP TABLE IF EXISTS Athlete;
DROP TABLE IF EXISTS Team;
DROP TABLE IF EXISTS Faculty;

CREATE TABLE Student
(
    netId VARCHAR(50) NOT NULL,
    name VARCHAR(50)  NOT NULL,
    PRIMARY KEY (netId)
);
CREATE TABLE Course
(
    crn INTEGER NOT NULL,
    courseName VARCHAR(50)  NOT NULL,
    PRIMARY KEY (crn)
);


CREATE TABLE Enroll
(
    netId VARCHAR(50) NOT NULL,
    crn INTEGER NOT NULL,
    PRIMARY KEY (netId, crn),
    FOREIGN KEY (netId) REFERENCES Student(netId),
    FOREIGN KEY (crn) REFERENCES Course(crn)
);

CREATE TABLE Team
(    
    teamName VARCHAR(30) NOT NULL,
    captainId VARCHAR(15) NULL,
    PRIMARY KEY (teamname)
);

CREATE TABLE Faculty(
 netId VARCHAR(10),
 name VARCHAR(10),
 mgrId VARCHAR(10)
);

Load some data

In [ ]:
%%sql
DELETE FROM Frequents;
DELETE FROM Likes;
DELETE FROM Serves;
DELETE FROM Enroll;
DELETE FROM Student;
DELETE FROM Course;
DELETE FROM Team;
DELETE FROM Faculty;

INSERT INTO Frequents(drinker, cafe) VALUES 
('Chris', 'A Cafe'),
('Chris', 'Double Trouble'),
('Risa', 'Brew Joint'),
('Risa', 'Java Lava'),
('Risa', 'Double Trouble');

INSERT INTO Likes(drinker, coffee) VALUES 
('Chris', 'Drip'),
('Chris', 'Espresso'),
('Risa', 'Cold Brew'),
('Risa', 'Drip');

INSERT INTO Serves(cafe, coffee) VALUES 
('A Cafe', 'Espresso'),
('A Cafe', 'Cold Brew'),
('Brew Joint', 'Espresso'),
('Double Trouble', 'Espresso'),
('Double Trouble', 'Cold Brew');

INSERT INTO Rates VALUES
('Risa', 'Cold Brew', 5),
('Risa', 'Drip', 3),
('Risa', 'Espresso', 4),
('Chris', 'Espresso', 2),
('Chris', 'Drip', 1),
('Ying', 'Drip', 1);


INSERT INTO Student(netId, name) VALUES
('rbm2', 'Risa'),
('abc1', 'Andre'),
('bcd2', 'Betty'),
('cde4', 'Chris');

INSERT INTO Course(crn, courseName) VALUES
(123, 'COMP 430'),
(234, 'COMP 533'),
(345, 'COMP 530');

INSERT INTO Enroll(crn, netId) VALUES
(123, 'abc1'),
(123, 'cde4'),
(345, 'abc1');

INSERT INTO Team(captainId, teamname) VALUES
('abc1', 'Tigers'),
('cde4', 'Knights'),
('msh2', 'Owls');


INSERT INTO Faculty(netId, name, mgrId) VALUES
('rbm2', 'Risa', 'abc1'),
('abc1', 'Andre', 'bcd2'),
('bcd2', 'Betty', 'cde4'),
('cde4', 'Chris', NULL);


Select clauses can also contain functions. One commonly used function is 

```NOW()``` 

which returns the current date and time.

Since this function doesn't draw from a relation, you don't even need a FROM clause:

In [ ]:
%%sql 
SELECT NOW()

### The FROM clause

In [ ]:
%%sql
SELECT f.*
FROM Frequents AS f

### The WHERE Clause

The WHERE clause can contain many things:
* Conditions
* JOIN conditions
* Selection (RA) clauses
* Subqueries
* ...

One useful clause uses ```LIKE``` which does a string pattern match. ```%``` is a wildcard that matches any number of characters.

Write a query to return the drinkers whose names start with 'C'

In [ ]:
%%sql
SELECT *
FROM Likes l
WHERE l.drinker -- your code here

Write a query that returns the Rates scores ```between``` 1 and 3

In [ ]:
%%sql
-- your code here


Here's an example of ```IN```. How does it work?

In [ ]:
%%sql
SELECT *
FROM Frequents f
WHERE f.drinker = 'Risa' AND f.cafe IN ('Double Trouble', 'Coffee House')

-- your thoughts here

## Joins

### CROSS JOIN

Cartesian product, returns all possible pairings

In [ ]:
%%sql
SELECT s.netId, c.crn
FROM Student S CROSS JOIN Course c

### INNER JOIN 

Who (name) has enrolled in which course(s) (name)?

In [ ]:
%%sql
SELECT s.name, c.courseName
FROM Student s INNER JOIN Enroll e ON s.netId = e.netId INNER JOIN Course c on e.crn = c.crn


Rewrite this query using natural joins

In [ ]:
%%sql
SELECT 
FROM 

Are the results any different? If so, in what way(s)?

your thoughts here

Rewrite this query using natural joins

In [ ]:
%%sql
-- your query here

Compare the inner join query to this version using a cross join

In [ ]:
%%sql
SELECT s.name, c.courseName
FROM Student s, Enroll e, Course c
WHERE s.netId = e.netId
  AND e.crn = c.crn

Are the results any different? If so, in what way(s)?

your thoughts here

### Left / Right  outer join

* Used to match up tuples from different relations
* Includes all the tuples from the "outer" side 
* If there is no matching tuple, assigns NULLs
* Returns a relation with all the attributes of R $\bullet$ all the attributes of S
* Tip: Pick one direction and use it consistently

Which students (name) haven't enrolled in any courses?

In [ ]:
%%sql
SELECT s.name
FROM Student s LEFT OUTER JOIN Enroll e ON s.netId = e.netId
WHERE e.crn IS NULL


Rewrite this query as a Right Outer Join

In [ ]:
%%sql
 



What does this expression represent?

In [ ]:
%%sql
SELECT *
FROM Enroll e RIGHT OUTER JOIN Course c ON e.crn = c.crn
WHERE e.crn IS NULL


your thoughts here

## Full Outer Join

Let's use the Student and Team tables for this example:

In [ ]:
%%sql
SELECT *
FROM Student;

In [ ]:
%%sql
SELECT *
FROM Team;

Full Outer Join example


In [ ]:
%%sql
SELECT *
FROM Student s FULL OUTER JOIN TEAM t ON s.netId = t.captainId


## Self Join

* Not a new type of join, but a special case
* Used to match up tuples from relation R back to itself
* Any type of JOIN may be used 
* Returns a relation with all the attributes of R $\bullet$ all the attributes of R

In [ ]:
%%sql
SELECT *
FROM Faculty;

In [ ]:
%%sql
SELECT f.name, Mgr.Name
FROM Faculty f JOIN Faculty Mgr ON f.mgrId = Mgr.netId

What does this expression represent?

A Every faculty member paired with every manager

B Every faculty member who has a manager, paired with that manager

## Practice queries

Who goes to a cafe serving Cold Brew? 

In [ ]:
%%sql
SELECT DISTINCT f.drinker
FROM Frequents AS f JOIN Serves AS s ON f.cafe = s.cafe 
WHERE  s.coffee = 'Cold Brew';


Try it without the 'DISTINCT'

In [ ]:
%%sql
SELECT f.drinker
FROM Frequents AS f JOIN Serves AS s ON f.cafe = s.cafe 
WHERE  s.coffee = 'Cold Brew';

What changed? Explain in the next cell.

your thoughts here

This query can be written another way, with the JOIN condition in the WHERE clause:

In [ ]:
%%sql
SELECT f.drinker 
FROM Frequents  AS f,  Serves  AS s 
WHERE f.cafe = s.cafe AND  s.coffee = 'Cold Brew'

Let's rename an attribute. 

Here, since the name we want to use has a space in it, we use double quotes.

In [ ]:
%%sql
SELECT DISTINCT f.drinker AS "Best Customers"
FROM Frequents  AS f JOIN Serves  AS s ON f.cafe = s.cafe 
WHERE  s.coffee = 'Cold Brew';

What is the ```AS``` doing? 

Note that "Best Customers" is in double quotes because of the space in the name. Typically, we use single quotes for text strings.

your thoughts here

What kind of join are we using?

A Cartesian product / Cross join

B Theta join

C Natural join

your thoughts here

### LIMIT and ORDER BY

In [ ]:
%%sql
SELECT r.score 
FROM Rates r
WHERE r.coffee = 'Cold Brew'
ORDER BY r.score DESC LIMIT 1


### The SELECT clause can contain many things.

Complete the following query to return ALL of the attributes from all of the relations

In [ ]:
%%sql
SELECT 
FROM Frequents  AS f,  Serves AS s
WHERE f.cafe = s.cafe AND  s.coffee = 'Cold Brew'

Next, return only the attributes from SERVES

In [ ]:
%%sql
SELECT 
FROM Frequents  AS f,  Serves AS s
WHERE f.cafe = s.cafe AND  s.coffee = 'Cold Brew'

Next,  add the text string 'Favorite' to the values returned.

Note that you use single quotes here, not double.

In [ ]:
%%sql
SELECT 
FROM Frequents  AS f,  Serves  AS s 
WHERE f.cafe = s.cafe AND  s.coffee = 'Cold Brew'